In [141]:
#import necessary packages
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.model_selection import train_test_split
from elosports.elo import Elo
import time
import matplotlib.pyplot as plt



In [142]:
PL_21_team_ratings = pd.read_csv('PL21_teamratings.csv')
PL2020_keepers = pd.read_csv('Data/2020-21/Players/PL2020_keepers.csv')
GK_Teams = pd.read_csv('Data/GK_Teams.csv', header=[1])

In [143]:
PL2020_keepers['matchweek'] = PL2020_keepers['matchweek'].str.slice(start=-3, stop=-1).astype(float)

In [144]:
GK_Teams['player'] = GK_Teams['Player'].str.slice(stop=-9)
GK_Teams.dropna(inplace=True)
GK_Teams.drop(columns='Player', inplace=True)
GK_Teams.drop(index=38, inplace=True)
GK_Teams

,Squad,player
1,Liverpool,Adrián
2,Arsenal,Rúnar Alex Rúnarsson
3,Liverpool,Alisson
4,Fulham,Alphonse Areola
5,Chelsea,Kepa Arrizabalaga
6,Crystal Palace,Jack Butland
7,West Brom,David Button
8,Chelsea,Willy Caballero
9,Manchester City,Scott Carson
10,Leeds United,Kiko Casilla


In [145]:
PL2020_gk = pd.merge(PL2020_keepers, GK_Teams, on='player', how='left')
PL2020_gk

,player,shots_on_target_against,goals_against_gk,saves,save_pct,psxg_gk,matchweek,Squad
0,Marek Rodák,5.0,3.0,2.0,40.0,2.5,1.0,Fulham
1,Bernd Leno,2.0,0.0,2.0,100.0,0.0,1.0,Arsenal
2,Vicente Guaita,5.0,0.0,5.0,100.0,0.8,1.0,Crystal Palace
3,Alex McCarthy,3.0,1.0,2.0,66.7,0.7,1.0,Southampton
4,Alisson,3.0,3.0,0.0,0.0,2.1,1.0,Liverpool
...,...,...,...,...,...,...,...,...
760,Robert Sánchez,4.0,2.0,2.0,50.0,1.1,38.0,Brighton
761,Alisson,4.0,0.0,4.0,100.0,1.1,38.0,Liverpool
762,Vicente Guaita,5.0,2.0,3.0,60.0,2.2,38.0,Crystal Palace
763,Marek Rodák,3.0,2.0,2.0,66.7,1.5,38.0,Fulham


In [146]:
PL2020_gk['Rating_gk'] = ((PL2020_gk['psxg_gk'] - PL2020_gk['goals_against_gk'])*10)+(PL2020_gk['saves']+3)+(PL2020_gk['save_pct'])
PL2020_gk['Home_id'] = PL2020_gk['Squad'] + PL2020_gk['matchweek'].astype(str)
PL2020_gk['Away_id'] = PL2020_gk['Squad'] + PL2020_gk['matchweek'].astype(str)
PL2020_gk['Rating_gk_home'] = PL2020_gk['Rating_gk']
PL2020_gk['Rating_gk_away'] = PL2020_gk['Rating_gk']
#PL2020_gk['Home_id'].duplicated()
PL2020_gk

,player,shots_on_target_against,goals_against_gk,saves,save_pct,psxg_gk,matchweek,Squad,Rating_gk,Home_id,Away_id,Rating_gk_home,Rating_gk_away
0,Marek Rodák,5.0,3.0,2.0,40.0,2.5,1.0,Fulham,40.0,Fulham1.0,Fulham1.0,40.0,40.0
1,Bernd Leno,2.0,0.0,2.0,100.0,0.0,1.0,Arsenal,105.0,Arsenal1.0,Arsenal1.0,105.0,105.0
2,Vicente Guaita,5.0,0.0,5.0,100.0,0.8,1.0,Crystal Palace,116.0,Crystal Palace1.0,Crystal Palace1.0,116.0,116.0
3,Alex McCarthy,3.0,1.0,2.0,66.7,0.7,1.0,Southampton,68.7,Southampton1.0,Southampton1.0,68.7,68.7
4,Alisson,3.0,3.0,0.0,0.0,2.1,1.0,Liverpool,-6.0,Liverpool1.0,Liverpool1.0,-6.0,-6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
760,Robert Sánchez,4.0,2.0,2.0,50.0,1.1,38.0,Brighton,46.0,Brighton38.0,Brighton38.0,46.0,46.0
761,Alisson,4.0,0.0,4.0,100.0,1.1,38.0,Liverpool,118.0,Liverpool38.0,Liverpool38.0,118.0,118.0
762,Vicente Guaita,5.0,2.0,3.0,60.0,2.2,38.0,Crystal Palace,68.0,Crystal Palace38.0,Crystal Palace38.0,68.0,68.0
763,Marek Rodák,3.0,2.0,2.0,66.7,1.5,38.0,Fulham,66.7,Fulham38.0,Fulham38.0,66.7,66.7


In [147]:
PL_21_team_ratings['Home_id'] = PL_21_team_ratings['HomeTeam'] + PL_21_team_ratings['Wk'].astype(str)
PL_21_team_ratings['Away_id'] = PL_21_team_ratings['AwayTeam'] + PL_21_team_ratings['Wk'].astype(str)

In [148]:
tmp = pd.merge(PL_21_team_ratings, PL2020_gk[['Home_id', 'Rating_gk_home']], on='Home_id', how='left')
out = pd.merge(tmp, PL2020_gk[['Away_id', 'Rating_gk_away']], on='Away_id', how='left')


In [149]:
def get_points(team, wk):
    for index, row in out.iterrows():
        #print(row[['HomeTeam','Wk']])
        if row['Wk'] == wk and row['HomeTeam'] == team:
            return row['Rating_gk_home']
        elif row['Wk'] == wk and row['AwayTeam'] == team:
            return row['Rating_gk_away']

def get_points_season(team, fr, to):
    out = 0
    for i in range(fr,to):
        out = out + get_points(team, i)

    return out


def append_form(timespan):
    formH = []
    formA = []
    for index, row in out.iterrows():
        wk = row['Wk']
        teamH = row['HomeTeam']
        teamA = row['AwayTeam']
        if int(wk) - timespan < 1:
            pointsH = get_points_season(teamH, 1, int(wk))
            pointsA = get_points_season(teamA, 1, int(wk))
        else:
            pointsH = get_points_season(teamH, int(wk)-timespan, int(wk))
            pointsA = get_points_season(teamA, int(wk)-timespan, int(wk))
        formH.append(pointsH)
        formA.append(pointsA)
    out['form_keeper_h'] = formH
    out['form_keeper_a'] = formA

append_form(5)

In [150]:
out.to_csv('PL20_allData.csv',index=False)

In [151]:
PL_20 = out[['Wk', 'HomeTeam', 'AwayTeam', 'FTR', 'FTHG', 'FTAG', 'Rating_Home', 'Rating_Away', 'Rating difference', 'form5GamesH', 'form5GamesA', 'form5GamesDifference', 'eloH', 'eloA', 'eloDifference', 'Rating_gk_home', 'Rating_gk_away', 'form_keeper_h', 'form_keeper_a']]
PL_20

,Wk,HomeTeam,AwayTeam,FTR,FTHG,FTAG,Rating_Home,Rating_Away,Rating difference,form5GamesH,form5GamesA,form5GamesDifference,eloH,eloA,eloDifference,Rating_gk_home,Rating_gk_away,form_keeper_h,form_keeper_a
0,1.0,Fulham,Arsenal,A,0,3,69.939394,76.212121,-6.272727,0,0,0,1500.000000,1500.000000,0.000000,40.0,105.0,0.0,0.0
1,1.0,Crystal Palace,Southampton,H,1,0,71.454545,71.303030,0.151515,0,0,0,1500.000000,1500.000000,0.000000,116.0,68.7,0.0,0.0
2,1.0,Liverpool,Leeds United,H,4,3,78.575758,67.969697,10.606061,0,0,0,1500.000000,1500.000000,0.000000,-6.0,52.0,0.0,0.0
3,1.0,West Ham,Newcastle Utd,A,0,2,71.303030,73.272727,-1.969697,0,0,0,1500.000000,1500.000000,0.000000,38.3,115.0,0.0,0.0
4,1.0,West Brom,Leicester City,A,0,3,68.718750,75.151515,-6.432765,0,0,0,1500.000000,1500.000000,0.000000,87.0,105.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
383,38.0,Liverpool,Crystal Palace,H,2,0,78.575758,71.454545,7.121212,13,6,7,1572.195622,1458.109252,114.086370,118.0,68.0,379.7,136.3
384,38.0,Manchester City,Everton,H,5,0,77.484848,74.393939,3.090909,9,9,0,1629.646004,1521.033816,108.612188,110.0,60.5,341.3,380.4
385,38.0,Sheffield Utd,Burnley,H,1,0,69.935484,70.166667,-0.231183,6,6,0,1362.194200,1452.510491,-90.316291,107.0,62.7,483.0,261.3
386,38.0,West Ham,Southampton,H,3,0,71.303030,71.303030,0.000000,7,4,3,1551.974233,1441.808453,110.165781,118.0,52.1,428.0,344.1


In [154]:
pd.set_option('display.max_rows', PL_20.shape[0]+1)
PL_20

,Wk,HomeTeam,AwayTeam,FTR,FTHG,FTAG,Rating_Home,Rating_Away,Rating difference,form5GamesH,form5GamesA,form5GamesDifference,eloH,eloA,eloDifference,Rating_gk_home,Rating_gk_away,form_keeper_h,form_keeper_a
0,1.0,Fulham,Arsenal,A,0,3,69.939394,76.212121,-6.272727,0,0,0,1500.000000,1500.000000,0.000000,40.0,1.050000e+02,0.0,0.0
1,1.0,Crystal Palace,Southampton,H,1,0,71.454545,71.303030,0.151515,0,0,0,1500.000000,1500.000000,0.000000,116.0,6.870000e+01,0.0,0.0
2,1.0,Liverpool,Leeds United,H,4,3,78.575758,67.969697,10.606061,0,0,0,1500.000000,1500.000000,0.000000,-6.0,5.200000e+01,0.0,0.0
3,1.0,West Ham,Newcastle Utd,A,0,2,71.303030,73.272727,-1.969697,0,0,0,1500.000000,1500.000000,0.000000,38.3,1.150000e+02,0.0,0.0
4,1.0,West Brom,Leicester City,A,0,3,68.718750,75.151515,-6.432765,0,0,0,1500.000000,1500.000000,0.000000,87.0,1.050000e+02,0.0,0.0
5,1.0,Tottenham,Everton,A,0,1,77.242424,74.393939,2.848485,0,0,0,1500.000000,1500.000000,0.000000,82.0,1.140000e+02,0.0,0.0
6,1.0,Brighton,Chelsea,A,1,3,70.787879,79.212121,-8.424242,0,0,0,1500.000000,1500.000000,0.000000,42.0,6.370000e+01,0.0,0.0
7,1.0,Sheffield Utd,Wolves,A,0,2,69.935484,69.636364,0.299120,0,0,0,1500.000000,1500.000000,0.000000,39.0,1.040000e+02,0.0,0.0
8,2.0,Everton,West Brom,H,5,2,74.393939,68.718750,5.675189,3,0,3,1512.801300,1487.198700,25.602600,40.0,2.460000e+01,114.0,87.0
9,2.0,Leeds United,Fulham,H,4,3,67.969697,69.939394,-1.969697,0,0,0,1492.801300,1487.198700,5.602600,56.0,4.600000e+01,52.0,40.0
